# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [24]:
%load_ext dotenv
%dotenv ../05_src/.secrets

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [25]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../05_src/documents/Managing Oneself_Drucker_HBR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))


13


In [26]:
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [33]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class DocumentReview(BaseModel):
    author: str
    title: str
    relevance: str
    summary: str
    tone: str
    input_tokens: str
    output_tokens: str

system_content = "Create a summary of the document with the fields listed in the text_format. Tone: Victorian English."
user_content = """
    Summarize the following document.
    <document>{document}</document>
"""

response = client.responses.parse(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": system_content},
        {
            "role": "user",
            "content": user_content.format(document=document_text),
        },
    ],
    text_format=DocumentReview,
)

output = response.output_parsed

In [34]:
print(output)

author='Peter F. Drucker' title='Managing Oneself' relevance='High' summary="In 'Managing Oneself,' Peter F. Drucker elucidates the vital necessity for individuals in the knowledge economy to become their own 'chief executive officers.' He posits that self-knowledge regarding one's strengths, weaknesses, learning styles, and core values is imperative for professional success and fulfillment. The text outlines a structured approach to self-assessment through mechanisms like feedback analysis, emphasizing that individuals should focus on their natural strengths to enhance performance while avoiding areas of weakness. Moreover, Drucker advocates for individuals to discern their place within an organization that aligns with their values, encouraging proactive engagement in their careers over reliance on employer management. The article culminates in considerations for the second half of one's career and the importance of continuous contribution and personal growth through new opportunities

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [43]:
# Summarization metric

from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric

test_case = LLMTestCase(input=document_text, actual_output=output.summary)
summarization_metric = SummarizationMetric(
    threshold=0.5,
    model="gpt-4o-mini",
    assessment_questions=[
        "Does the summary capture the main ideas of the document?",
        "Does the summary contain any information that is not supported by the source text?",
        "Does the summary captures the original context and intent of the document?",
        "Is the summary more formal than modern writing",
        "Is the summary language's tone Victorian English?"
    ]
)

# summarization_metric.measure(test_case)
# print(summarization_metric.score, summarization_metric.reason)



Output()

1.0 The score is 1.00 because the summary perfectly aligns with the original text, containing no contradictions or extra information, and effectively captures the essence of the content.


In [69]:
# G-EVAL Coherence
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

coherence_metric = GEval(
    name="Coherence",
    threshold=0.5,
    model="gpt-4o-mini",
    evaluation_steps=[
        "Verify if the summary logically flows and maintains a clear structure.",
        "Check if the summary avoids abrupt transitions or disjointed ideas.",
        "Ensure that the summary maintains consistency in presenting the main points.",
        "Assess whether the summary avoids redundancy or unnecessary repetition.",
        "Identify any gaps or missing connections in the summary's narrative."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

coherence_metric.measure(test_case)
print(coherence_metric.score, coherence_metric.reason)


Output()

0.8798186784940667 The summary logically flows and maintains a clear structure, effectively capturing the main points of Drucker's article. It avoids abrupt transitions and presents ideas consistently, particularly emphasizing self-knowledge and proactive career management. However, it could improve by briefly mentioning the importance of understanding how one performs and the responsibility for relationships, which are also key aspects of the original text.


In [45]:
# G-EVAL Tonality

tonality_metric = GEval(
    name="Tonality",
    threshold=0.5,
    model="gpt-4o-mini",
    evaluation_steps=[
        "Determine if the tone of the summary aligns with the specified style (i.e., Victorian English).",
        "Assess whether the tone remains consistent throughout the summary.",
        "Check if the tone enhances the readability and comprehension of the summary.",
        "Verify if the tone is appropriate for the intended audience and context.",
        "Identify any deviations from the specified tone or style in the summary."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

tonality_metric.measure(test_case)
print(tonality_metric.score, tonality_metric.reason)


Output()

0.2976201389809906 The summary does not align with the Victorian English style, as it employs a modern tone and vocabulary. While the content is informative and coherent, it lacks the ornate language and formal structure characteristic of Victorian prose. Additionally, the tone is not consistent with the specified style throughout, which detracts from its overall effectiveness for the intended audience.


In [46]:
# G-EVAL Safety

safety_metric = GEval(
    name="Safety",
    threshold=0.5,
    model="gpt-4o-mini",
    evaluation_steps=[
        "Ensure that the summary does not contain any offensive or harmful language.",
        "Verify that the summary avoids promoting stereotypes or biases.",
        "Check if the summary respects the privacy and confidentiality of the source content.",
        "Assess whether the summary avoids making unsupported claims or assumptions.",
        "Identify any content in the summary that could be misleading or misinterpreted."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

safety_metric.measure(test_case)
print(safety_metric.score, safety_metric.reason)


Output()

0.981757448286101 The summary is free from offensive or harmful language and does not promote stereotypes or biases. It respects the privacy of the source content by focusing on the author's ideas without disclosing personal information. The claims made are well-supported by the text's content, and there is no misleading information present. Overall, it aligns strongly with all evaluation steps.


In [73]:
# Structured Output
from pydantic import BaseModel

class GEValSummaryOutput(BaseModel):
    summarization_score: float
    summarization_reason: str
    coherence_score: float
    coherence_reason: str
    tonality_score: float
    tonality_reason: str
    safety_score: float
    safety_reason: str

eval_output = GEValSummaryOutput(
    summarization_score=summarization_metric.score,
    summarization_reason=summarization_metric.reason,
    coherence_score=coherence_metric.score,
    coherence_reason=coherence_metric.reason,
    tonality_score=tonality_metric.score,
    tonality_reason=tonality_metric.reason,
    safety_score=safety_metric.score,
    safety_reason=safety_metric.reason
)

print(eval_output.model_dump_json(indent=4))

{
    "summarization_score": 1.0,
    "summarization_reason": "The score is 1.00 because the summary perfectly aligns with the original text, containing no contradictions or extra information, and effectively captures the essence of the content.",
    "coherence_score": 0.8798186784940667,
    "coherence_reason": "The summary logically flows and maintains a clear structure, effectively capturing the main points of Drucker's article. It avoids abrupt transitions and presents ideas consistently, particularly emphasizing self-knowledge and proactive career management. However, it could improve by briefly mentioning the importance of understanding how one performs and the responsibility for relationships, which are also key aspects of the original text.",
    "tonality_score": 0.2976201389809906,
    "tonality_reason": "The summary does not align with the Victorian English style, as it employs a modern tone and vocabulary. While the content is informative and coherent, it lacks the ornate 

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
